# Movielens 영화 추천

데이터셋: Movielens 데이터
- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있다.
   - MovieLens 1M Dataset 사용 권장
- 별점 데이터는 대표적인 explicit 데이터
   - 하지만, implicit 데이터로 간주하고 테스트해 볼 수 있다.
- 별점을 시청횟수로 해석해서 생각한다.
   - 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외한다.

In [1]:
import os
import scipy
import implicit
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


## 데이터 준비와 전처리

In [2]:
rating_file_path = os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### 3점 이상만 남기기

In [3]:
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


### ratings 컬럼의 이름을 counts로 바꾸기

In [4]:
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

### 영화 제목을 보기 위해 메타 데이터를 읽어오기

In [6]:
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

## 데이터 분석
ratings에 있는 유니크한 영화 개수  
ratings에 있는 유니크한 사용자 수  
가장 인기 있는 영화 30개(인기순)  

### 유니크한 영화 개수

In [7]:
unique_movies = ratings['movie_id'].nunique()
print(f'유니크한 영화 개수: {unique_movies}')

유니크한 영화 개수: 3628


### 유니크한 사용자 수

In [8]:
unique_users = ratings['user_id'].nunique()
print(f'유니크한 사용자 수: {unique_users}')

유니크한 사용자 수: 6039


### 가장 인기 있는 영화 30개 (인기순)

In [9]:
movie_popularity = ratings.groupby('movie_id')['user_id'].count().reset_index(name='count').merge(movies, on='movie_id').sort_values('count', ascending=False).head(30)
print(movie_popularity[['title', 'count']])

                                                  title  count
2600                             American Beauty (1999)   3211
249           Star Wars: Episode IV - A New Hope (1977)   2910
1080  Star Wars: Episode V - The Empire Strikes Back...   2885
1094  Star Wars: Episode VI - Return of the Jedi (1983)   2716
1810                         Saving Private Ryan (1998)   2561
569                   Terminator 2: Judgment Day (1991)   2509
573                    Silence of the Lambs, The (1991)   2498
1082                     Raiders of the Lost Ark (1981)   2473
1152                          Back to the Future (1985)   2460
2325                                 Matrix, The (1999)   2434
462                                Jurassic Park (1993)   2413
2507                            Sixth Sense, The (1999)   2385
587                                        Fargo (1996)   2371
106                                   Braveheart (1995)   2314
1419                                Men in Black (1997)

## 모델 구축 및 훈련

### 내가 선호하는 영화를 5가지 골라서 ratings에 추가

####  좋아하는 영화 고르기
데이터셋에 있는 영화 제목을 정확하게 사용해야 한다.

In [10]:
my_favorite_movies = ['Toy Story (1995)', 
                      'Jumanji (1995)', 
                      'Men in Black (1997)', 
                      'Heat (1995)', 
                      'Sabrina (1995)']

#### ratings 추가
- 'minji'이라는 user_id가 위 영화들을 모두 5점을 줬다고 가정
    - 이를 데이터프레임에 추가하기 위해 먼저 movie_id를 찾아야 한다.
- 기존 ratings 데이터프레임에 추가

In [11]:
my_movies = pd.DataFrame({
    'user_id': ['minji'] * 5,
    'movie_id': movies[movies['title'].isin(my_favorite_movies)]['movie_id'].tolist(),
    'counts': [5] * 5,
    'timestamp': [0] * 5
})

In [12]:
if not ratings.isin({'user_id':['minji']})['user_id'].any():  # user_id에 'minji'이라는 데이터가 없다면
    ratings = ratings.append(my_movies)                       # 위에 임의로 만든 my_movies 데이터를 추가해 준다.

### 데이터 인덱싱
사용자 ID와 영화 ID를 순차적인 정수 인덱스로 매핑한다.  
이는 모델이 인식할 수 있는 형태로 데이터를 변환하는 과정이다.

#### 고유한 사용자와 영화를 찾아내는 과정

In [13]:
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

In [14]:
print('user unique:', user_unique)
print('movie unique: ', movie_unique)

user unique: [1 2 3 ... 6039 6040 'minji']
movie unique:  [1193  661  914 ...  690 2909 1360]


#### 사용자와 영화 indexing

In [15]:
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

#### 데이터 컬럼 내 값을 인덱싱 값으로 변경

In [16]:
ratings['user_id'] = ratings['user_id'].map(user_to_idx.get)
ratings['movie_id'] = ratings['movie_id'].map(movie_to_idx.get)

In [17]:
print(user_to_idx['minji'])                           # 새로 추가한 사용자 'minji'의 인덱스 확인
print(movie_to_idx[my_movies.iloc[0]['movie_id']])    # 'Toy Story (1995)'의 인덱스 확인

6039
40


### CSR matrix

#### 데이터프레임에서 필요한 컬럼만 추출
사용자 ID와 영화 ID는 이미 인덱싱된 값으로 변환되었다.

In [18]:
user_ids = ratings['user_id'].values
movie_ids = ratings['movie_id'].values
counts = ratings['counts'].values

#### CSR Matrix 생성
행렬의 크기는 사용자 수와 영화 수의 최대값에 +1을 한 값으로 설정한다.(인덱스는 0부터 시작하기 때문)

In [19]:
num_users = ratings['user_id'].nunique()
num_movies = ratings['movie_id'].nunique()

csr_data = csr_matrix((counts, (user_ids, movie_ids)), shape=(num_users + 1, num_movies + 1))

In [20]:
print(csr_data)

  (0, 0)	5
  (0, 1)	3
  (0, 2)	3
  (0, 3)	4
  (0, 4)	5
  (0, 5)	3
  (0, 6)	5
  (0, 7)	5
  (0, 8)	4
  (0, 9)	4
  (0, 10)	5
  (0, 11)	4
  (0, 12)	4
  (0, 13)	4
  (0, 14)	5
  (0, 15)	4
  (0, 16)	3
  (0, 17)	4
  (0, 18)	5
  (0, 19)	4
  (0, 20)	3
  (0, 21)	3
  (0, 22)	5
  (0, 23)	5
  (0, 24)	3
  :	:
  (6038, 2311)	4
  (6038, 2317)	5
  (6038, 2386)	4
  (6038, 2394)	5
  (6038, 2424)	4
  (6038, 2437)	4
  (6038, 2446)	5
  (6038, 2471)	4
  (6038, 2511)	5
  (6038, 2523)	4
  (6038, 2559)	3
  (6038, 2560)	4
  (6038, 2631)	5
  (6038, 2648)	4
  (6038, 2654)	5
  (6038, 2738)	4
  (6038, 2740)	5
  (6038, 2857)	5
  (6038, 2860)	3
  (6038, 3311)	5
  (6039, 40)	5
  (6039, 175)	5
  (6039, 373)	5
  (6039, 513)	5
  (6039, 516)	5


### 모델 구축
`als_model = AlternatingLeastSquares` 모델을 직접 구성하여 훈련시키기


```python
# 초기 설정
als_model = AlternatingLeastSquares(factors=100, 
                                    regularization=0.01, 
                                    use_gpu=False, 
                                    iterations=20, 
                                    dtype=np.float32)

```

In [39]:
als_model = AlternatingLeastSquares(factors=500, 
                                    regularization=0.001, 
                                    use_gpu=False, 
                                    iterations=100, 
                                    dtype=np.float32)

#### 모델 훈련을 위한 데이터 준비
implicit 라이브러리는 item * user 형태의 매트릭스를 기대한다.

In [40]:
csr_data_transpose = csr_data.T
print(csr_data_transpose)

  (0, 0)	5
  (1, 0)	3
  (2, 0)	3
  (3, 0)	4
  (4, 0)	5
  (5, 0)	3
  (6, 0)	5
  (7, 0)	5
  (8, 0)	4
  (9, 0)	4
  (10, 0)	5
  (11, 0)	4
  (12, 0)	4
  (13, 0)	4
  (14, 0)	5
  (15, 0)	4
  (16, 0)	3
  (17, 0)	4
  (18, 0)	5
  (19, 0)	4
  (20, 0)	3
  (21, 0)	3
  (22, 0)	5
  (23, 0)	5
  (24, 0)	3
  :	:
  (2311, 6038)	4
  (2317, 6038)	5
  (2386, 6038)	4
  (2394, 6038)	5
  (2424, 6038)	4
  (2437, 6038)	4
  (2446, 6038)	5
  (2471, 6038)	4
  (2511, 6038)	5
  (2523, 6038)	4
  (2559, 6038)	3
  (2560, 6038)	4
  (2631, 6038)	5
  (2648, 6038)	4
  (2654, 6038)	5
  (2738, 6038)	4
  (2740, 6038)	5
  (2857, 6038)	5
  (2860, 6038)	3
  (3311, 6038)	5
  (40, 6039)	5
  (175, 6039)	5
  (373, 6039)	5
  (513, 6039)	5
  (516, 6039)	5


### 모델 훈련

In [41]:
als_model.fit(csr_data_transpose)

  0%|          | 0/100 [00:00<?, ?it/s]

## 선호도 파악
내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도 파악

### 인덱스 가져오기

In [42]:
# 'minji' 사용자 인덱스
minji_idx = user_to_idx['minji']
minji_idx

6039

In [43]:
# 'Toy Story (1995)' 영화 인덱스
toy_story_idx = movie_to_idx[1] # 영화 ID가 1인 'Toy Story (1995)'
toy_story_idx

40

### 모델을 사용하여 'minji'의 'Toy Story (1995)'에 대한 선호도 파악

In [44]:
minji_toy_story_pref = np.dot(als_model.user_factors[minji_idx], als_model.item_factors[toy_story_idx])
print('minji의 Toy Story (1995)에 대한 선호도:', minji_toy_story_pref)

minji의 Toy Story (1995)에 대한 선호도: 0.9340021


### 다른 영화에 대한 선호도 파악

In [45]:
# ex) 'Grumpier Old Men (1995)' 영화의 인덱스 가져오기
grumpier_old_men_idx = movie_to_idx[3] # 영화 ID가 3인 'Grumpier Old Men (1995)'

# 모델을 사용하여 'minji'의 'Grumpier Old Men (1995)'에 대한 선호도 파악
minji_grumpier_old_men_pref = als_model.user_factors[minji_idx].dot(als_model.item_factors[grumpier_old_men_idx])
print('minji의 Grumpier Old Men (1995)에 대한 선호도:', minji_grumpier_old_men_pref)

minji의 Grumpier Old Men (1995)에 대한 선호도: -0.0027952096


## 비슷한 영화 추천 받기
내가 좋아하는 영화와 비슷한 영화를 추천받기

In [46]:
# 'Toy Story (1995)'와 비슷한 영화 찾기
similar_movies = als_model.similar_items(toy_story_idx, N=5)
for movie_id, similarity in similar_movies:
    print(movies[movies['movie_id'] == movie_id]['title'].values[0], similarity)

Cry, the Beloved Country (1995) 1.0
Man Facing Southeast (Hombre Mirando al Sudeste) (1986) 0.44462755
Man Bites Dog (C'est arrivé près de chez vous) (1992) 0.43920505
Kill, Baby... Kill! (Operazione Paura) (1966) 0.4377068
Howards End (1992) 0.4367935


## 좋아할 만한 영화 추천 받기
내가 가장 좋아할 만한 영화들을 추천받기

In [47]:
# 'minji'에게 영화 추천하기
recommended_movies = als_model.recommend(minji_idx, csr_data, N=5, filter_already_liked_items=True)
for movie_id, score in recommended_movies:
    print(movies[movies['movie_id'] == movie_id]['title'].values[0], score)

Howling, The (1980) 0.20604476
Black Sheep (1996) 0.19522905
Judge Dredd (1995) 0.1879754
Walking Dead, The (1995) 0.17447287
Usual Suspects, The (1995) 0.13807811


## 결과

### 선호도 파악
- 'Toy Story (1995)'에 대한 선호도: 0.9340021
    - 'minji'가 'Toy Story (1995)'를 매우 좋아할 것이라는 모델의 예측을 나타낸다. 
        - 선호도 점수는 사용자의 특성 벡터와 아이템의 특성 벡터를 내적(dot product)한 값으로, 두 벡터가 얼마나 유사한지를 나타낸다. 
        - 높은 점수는 높은 선호도를 의미한다.

- 'Grumpier Old Men (1995)'에 대한 선호도: -0.0027952096
    - 거의 0에 가깝거나 약간 부정적인 값을 가지고 있다.
    - 'minji'가 이 영화에 대해 별로 관심이 없거나, 싫어할 가능성이 있다는 것을 의미한다.


### 비슷한 영화 추천
- 비슷한 영화 목록: 'Toy Story (1995)'와 비슷한 영화 추천
    - 추천된 영화들은 'Toy Story'와 유사한 특성 벡터를 가진 영화들이다. 
    - 그러나 추천된 영화들의 제목을 보면 'Toy Story'와 직접적인 관련성이 떨어지는 것으로 보인다.
        - 모델이 영화의 장르나 내용보다 사용자의 상호작용 패턴에 기반하여 유사도를 계산하기 때문에 발생할 수 있다. 
        - 또한, 데이터의 희소성이나 모델 파라미터 설정에 따라 추천의 질이 달라질 수 있다.
        
### 개인화된 영화 추천
- 개인화된 영화 목록: 'minji'에게 추천할 영화 목록 
    - 추천 목록은 'minji'의 과거 상호작용(평점 데이터)을 기반으로 생성된다. 
        - 추천된 영화들은 모델이 'minji'가 관심을 가질 것으로 예측한 영화이다. 
    - 'Usual Suspects, The (1995)'와 같은 영화는 더 높은 점수를 받았으며, 이는 'minji'의 취향과 더 일치할 가능성이 높음을 의미한다.

### 결론
종합적으로, 이 모델은 사용자의 과거 상호작용을 기반으로 영화에 대한 개인별 선호도를 예측하고, 그 결과를 바탕으로 개인화된 영화 추천을 제공한다. 하지만, 모델의 추천이 항상 사용자의 실제 취향을 정확히 반영하는 것은 아니며, 모델의 성능은 사용된 데이터, 하이퍼파라미터, 알고리즘의 특성 등 다양한 요소에 의해 영향을 받는다. 따라서, 추천 시스템의 성능을 향상시키기 위해서는 지속적인 실험과 최적화가 필요하다.

## 회고
이 프로젝트는 실제 사용자의 선호도 데이터를 기반으로 하는 추천 시스템의 구현을 통해,   
데이터 과학과 머신러닝의 실질적인 적용 사례를 경험할 수 있는 좋은 기회였다.   
특히, AlternatingLeastSquares 모델을 활용하여 사용자와 아이템 간의 상호작용을 학습하고,   
이를 기반으로 추천을 수행하는 과정은 매우 흥미로웠다.  

실험을 통해 찾은 하이퍼파라미터 설정은 시작점에 불과하다.   
더 정밀한 튜닝과 교차 검증을 통해 모델의 일반화 성능을 향상시킬 필요가 있는 거 같다.  
영화 추천 뿐만 아니라, 리그 오브 레전드 같은 게임 추천 시스템 구현에도 관심을 가지고,  
다양한 도메인의 추천 시스템에 대한 이해와 경험을 넓힐 수 있었다.  

이 프로젝트를 통해 추천 시스템의 기본적인 원리와 구현 방법을 이해할 수 있었으며,   
실제 데이터를 사용한 실습을 통해 이론적 지식을 실제 문제에 적용하는 경험을 할 수 있었다.   
추천 시스템은 다양한 온라인 서비스에서 중요한 역할을 하고 있으며, 이 분야의 기술이 지속적으로 발전하고 있다.   
앞으로도 새로운 기술과 알고리즘을 학습하고, 다양한 도메인에 적용해보는 것을 통해 지속적으로 성장해 나가고자 한다.